In [1]:
from database.SQLDBConnector import SQLDBConnector
from utils.query import async_process_df, extract_gest_age

from datetime import datetime

In [2]:
mobile = '18810833167'

query = f"""
SELECT
FROM_UNIXTIME(r.start_ts) as measurement_date,
r.contraction_url,
r.hb_baby_url,
r.raw_fetal_url,
r.basic_info,
r.conclusion
FROM
extant_future_user.user AS uu
INNER JOIN
origin_data_record AS r
ON uu.id = r.user_id
AND r.contraction_url <> ''
AND r.hb_baby_url <> ''
AND uu.mobile = '{mobile}'
AND r.start_ts BETWEEN UNIX_TIMESTAMP('2025-03-01 00:00:00') AND UNIX_TIMESTAMP('{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
;
"""

In [3]:
sql = SQLDBConnector()

df = sql.query_to_dataframe(query=query)

print(len(df))

uc_results, fhr_results, fmov_results = await async_process_df(df)

sorted_uc_list      = sorted(uc_results, key=lambda x: x[0])
sorted_fhr_list     = sorted(fhr_results, key=lambda x: x[0])
sorted_fmov_list    = sorted(fmov_results, key=lambda x: x[0])

count = 0 ; dates = set()
rows = []
for idx, row in df.iterrows():

    # Extract UC, FHR data ; Do not filter by < 20 minutes yet
    uc_data     = sorted_uc_list[idx][1].split("\n")
    fhr_data    = sorted_fhr_list[idx][1].split("\n")

    # Extract gestational age
    conclusion = row['conclusion'] ; basic_info = row['basic_info']
    gest_age = extract_gest_age(conclusion, basic_info)

    # Check if UC/FHR are both >= 20 minutes
    if len(uc_data) < 60*20 or len(fhr_data) < 60*20:
        continue

    # Check if gestational age is present
    if gest_age is None:
        continue

    count += 1
    dates.add(row['measurement_date'].strftime("%Y-%m-%d"))

    rows.append(row)

rows.sort(key=lambda x: x['measurement_date'])

print(f"{mobile} : {count} measurements across {len(dates)} unique dates")
print(f"Earliest : {rows[0]['measurement_date']}")
print(f"Latest   : {rows[-1]['measurement_date']}")

SSH TUNNEL STARTED ON PORT 52157
29
18810833167 : 28 measurements across 22 unique dates
Earliest : 2025-10-25 19:48:48
Latest   : 2025-11-18 19:37:30
